In [1]:
import pandas as pd
import os
from estnltk.converters.conll.conll_importer import conll_to_text
from estnltk_core.layer_operations import split_by_sentences
import estnltk
from estnltk import Text
from tqdm import tqdm

In [ ]:
# mis on fraasi peasõna ja verb

In [2]:
def get_peasona_and_verb(input_file, sent, phrase):
    text_obj = conll_to_text( input_file, syntax_layer='stanza_syntax' )
    texts2 = split_by_sentences(text=text_obj,
                                   layers_to_keep=list(text_obj.layers),
                                   trim_overlapping=True
                                   )
    # target text object for the sentence
    target = None
    target_idx = None
    for i,txt in enumerate(texts2):
        #clean_txt = clean(txt.text)
        if txt.text == sent:
            target = txt
            target_idx = i
    if target_idx is None:
        for i,txt in enumerate(texts2):
            clean_txt = clean(txt.text)
            clean_sent = clean(sent)
            if clean_txt == clean_sent:
                target = txt
                target_idx = i
    
    # target layer
    target_layer = texts2[target_idx]["stanza_syntax"]
    #display(target_layer)
    
    # words in the removed phrase
    phrase_words = {}
    for word in phrase.split(" "):
        phrase_words[word] = None
    # and corresponding indexes for the phrase words
    for idx, span in enumerate(target_layer):
        if span.text in phrase_words.keys():
            phrase_words[span.text] = idx
    # heads for the phrase words
    phrase_words_heads = {}
    for word in phrase_words.keys():
        phrase_words_heads[word] = target_layer[phrase_words[word]].head
        
    # word with the smallest head is the main word in phrase
    peasona = min(phrase_words_heads, key=phrase_words_heads.get)
    peasona_idx = phrase_words[peasona]
    peasonahead = phrase_words_heads[peasona]
    
    ## verbi otsimine
    phrase_verb = None
    isverb = False
    if peasonahead != 0:
        # peasona on verb
        if target_layer[peasona_idx].xpostag == "V":
            #print("peasona on verb")
            phrase_verb = peasona_idx
            isverb = True

        # peasona head on verb
        elif target_layer[peasonahead-1].xpostag == "V":
            phrase_verb = peasonahead
            isverb = True
            #print("head on verb", phrase_verb)

        # peasona head ei ole verb, tuleb hakata ülespoole liikuma
        elif target_layer[peasonahead-1].xpostag != "V":
            new_head = peasonahead
            phrase_verb = None
            while not isverb:
                #print("peasõna head", new_head, "ehk ", target_layer[new_head-1])
                if target_layer[new_head-1].xpostag == "V":
                    isverb = True
                    phrase_verb = new_head
                else:
                    new_head = target_layer[new_head-1].head
                    if new_head == 0 and isverb==False: # kui jõudsime rootini aga verbi ei ole
                        break

    # fraasi peasõna ei ole verbi alluv, võta siis esimene verb
    if not isverb or peasonahead == 0:
        for span in target_layer:
            if span.upostag == "VERB" or span.xpostag == "V":
                phrase_verb = span.id
                break
    
    verb = None
    if phrase_verb is not None:
        verb = target_layer[phrase_verb-1].text
    
    
    # return peasõna, peasõna index, peasõna parent ehk head, verb, verbi id
    return peasona, peasona_idx, peasonahead, verb, phrase_verb

In [4]:
deprel = "obl"
data = pd.read_csv(f"data/{deprel}_all_hand_annotated.csv", sep=";", encoding="utf-8")
root = r".../UDpuupank/UD2_11_udreposse"

In [5]:
free_sent_data = []

for i in tqdm(range(len(data))):
    sent = data.iloc[i]["sentence"]
    phrase = data.iloc[i]["removed"]
    file = data.iloc[i]["fpath"]
    input_file = os.path.join(root, file) 
    free_sent_data.append((get_peasona_and_verb(input_file, sent, phrase)))

100%|█████████████████████████████████████| 2744/2744 [4:13:54<00:00,  5.55s/it]


In [6]:
data["sentence_analysis_data"] = free_sent_data

In [7]:
free_peasonad = []
free_verbs = []

for data2 in free_sent_data:
    free_peasonad.append(data2[0])
    free_verbs.append(data2[3])

In [8]:
data["peasona"] = free_peasonad
data["verb"] = free_verbs

In [10]:
data

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la,sentence_analysis_data,peasona,verb
0,1,Dev/aja_luup200009_ud211.conllu,"Esimesel korral oli vene rahvas sõja vastu , aga seekord ollakse siiski poolt .",0,15,Esimesel korral,bound,"oli vene rahvas sõja vastu , aga seekord ollakse siiski poolt .",100.0,100.0,100.0,"(Esimesel, 0, 2, oli, 3)",Esimesel,oli
1,3,Train/aja_ee199920_osa_1_ud211.conllu,Suusakeskus hakkas poolest talvest tööle ja muutus üsna ruttu populaarseks .,35,40,tööle,bound,Suusakeskus hakkas poolest talvest ja muutus üsna ruttu populaarseks .,90.0,90.0,90.0,"(tööle, 4, 2, hakkas, 2)",tööle,hakkas
2,11,Train/tea_geofyysika.conllu,"Selles kunstis ei saanud nähtavasti keegi Aristotelese ( 384-322 e. Kr. ) vastu , kes oskas erakordse osavusega ükskõik millise oponendi väited maatasa teha , et seejärel omad kui ainuõiged esile tuua .",42,200,"Aristotelese ( 384-322 e. Kr. ) vastu , kes oskas erakordse osavusega ükskõik millise oponendi väited maatasa teha , et seejärel omad kui ainuõiged esile tuua",bound,Selles kunstis ei saanud nähtavasti keegi .,100.0,100.0,100.0,"(Aristotelese, 6, 4, saanud, 4)",Aristotelese,saanud
3,17,Train/aja_luup200202_osa_3_ud211.conllu,"Erastamise lõppedes selgus , et insaiderite kätte oli libisenud keskmiselt 66 protsenti ettevõtte aktsiatest ...",0,10,Erastamise,bound,"lõppedes selgus , et insaiderite kätte oli libisenud keskmiselt 66 protsenti ettevõtte aktsiatest ...",92.9,92.9,100.0,"(Erastamise, 0, 2, lõppedes, 2)",Erastamise,lõppedes
4,18,Train/aja_luup200202_osa_3_ud211.conllu,"Erastamise lõppedes selgus , et insaiderite kätte oli libisenud keskmiselt 66 protsenti ettevõtte aktsiatest ...",32,49,insaiderite kätte,bound,"Erastamise lõppedes selgus , et oli libisenud keskmiselt 66 protsenti ettevõtte aktsiatest ...",92.3,92.3,100.0,"(kätte, 6, 6, libisenud, 9)",kätte,libisenud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,2730,Train/aja_sloleht20071217_ud211.conllu,"6. augustil 1997 ilmus ühes ajalehes foto , kus Dodi ja Diana suudlevad .",0,16,6. augustil 1997,unnatural,"ilmus ühes ajalehes foto , kus Dodi ja Diana suudlevad .",81.8,100.0,81.8,"(6., 0, 2, ilmus, 4)",6.,ilmus
2740,2741,Train/ilu_remsu_ud211.conllu,"Euroopasse jõudis Hiina siid 2. sajandil pärast Kristust , läks veel mõnisada aastat ning araablaste kaudu jõudsid Euroopasse siidiussid ning oskus neid kasvatada .",0,10,Euroopasse,unnatural,"jõudis Hiina siid 2. sajandil pärast Kristust , läks veel mõnisada aastat ning araablaste kaudu jõudsid Euroopasse siidiussid ning oskus neid kasvatada .",91.3,91.3,95.7,"(Euroopasse, 17, 17, jõudsid, 17)",Euroopasse,jõudsid
2741,2742,Train/ilu_remsu_ud211.conllu,"Euroopasse jõudis Hiina siid 2. sajandil pärast Kristust , läks veel mõnisada aastat ning araablaste kaudu jõudsid Euroopasse siidiussid ning oskus neid kasvatada .",29,56,2. sajandil pärast Kristust,unnatural,"Euroopasse jõudis Hiina siid , läks veel mõnisada aastat ning araablaste kaudu jõudsid Euroopasse siidiussid ning oskus neid kasvatada .",95.0,95.0,100.0,"(sajandil, 5, 2, jõudis, 2)",sajandil,jõudis
2742,2744,Train/ilu_remsu_ud211.conllu,"Euroopasse jõudis Hiina siid 2. sajandil pärast Kristust , läks veel mõnisada aastat ning araablaste kaudu jõudsid Euroopasse siidiussid ning oskus neid kasvatada .",90,106,araablaste kaudu,unnatural,"Euroopasse jõudis Hiina siid 2. sajandil pärast Kristust , läks veel mõnisada aastat ning jõudsid Euroopasse siidiussid ning oskus neid kasvatada .",86.4,86.4,95.5,"(kaudu, 15, 15, jõudsid, 17)",kaudu,jõudsid


In [11]:
data.to_csv(f"data/{deprel}_all_hand_annotated_peasona_verb.csv", index=False, sep=";", encoding='utf-8')

In [9]:
#data[data["verb"].isna()]

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la,sentence_analysis_data,peasona,verb
319,765,Train/tea_dr8047.conllu,"Vastkoorunud tibude aju ja maksa rasvhappeline koostis sõltuvalt kanade söödas sisaldunud õlidest , % üldlipiididest ( Cherian , Sim , 1997 )",65,78,kanade söödas,bound,"Vastkoorunud tibude aju ja maksa rasvhappeline koostis sõltuvalt sisaldunud õlidest , % üldlipiididest ( Cherian , Sim , 1997 )",90.0,95.0,95.0,"(kanade, 8, 10, None, None)",kanade,None
320,766,Train/tea_dr8047.conllu,"Vastkoorunud tibude aju ja maksa rasvhappeline koostis sõltuvalt kanade söödas sisaldunud õlidest , % üldlipiididest ( Cherian , Sim , 1997 )",65,97,kanade söödas sisaldunud õlidest,bound,"Vastkoorunud tibude aju ja maksa rasvhappeline koostis sõltuvalt , % üldlipiididest ( Cherian , Sim , 1997 )",88.9,94.4,94.4,"(õlidest, 11, 8, None, None)",õlidest,None
660,1481,Train/aja_pm20001004_osa_2_ud211.conllu,"Kindlasti mitte puhta tõu paleuse tagamist , rahvusriiki kui ideaali või mitte-eestlastesse vaenulikku hoiakut .",73,91,mitte-eestlastesse,bound,"Kindlasti mitte puhta tõu paleuse tagamist , rahvusriiki kui ideaali või vaenulikku hoiakut .",92.9,92.9,100.0,"(mitte-eestlastesse, 11, 13, None, None)",mitte-eestlastesse,None
1018,2252,Train/aja_luup200106_osa_3_ud211.conllu,KT : Võrdse tahtmise korral kindlasti .,5,27,Võrdse tahtmise korral,bound,KT : kindlasti .,75.0,100.0,75.0,"(Võrdse, 2, 4, None, None)",Võrdse,None
1154,2535,Train/tea_toohoive.conllu,"Hõivatud uuringunädalal tavalisest vähem töötamise põhjuste järgi ,",24,34,tavalisest,bound,"Hõivatud uuringunädalal vähem töötamise põhjuste järgi ,",85.7,100.0,85.7,"(tavalisest, 2, 4, None, None)",tavalisest,None
1391,261,Train/tea_toohoive.conllu,"Uuringu nädalal töötatud keskmine tundide arv ( hõivatud ) ,",0,15,Uuringu nädalal,free,"töötatud keskmine tundide arv ( hõivatud ) ,",87.5,87.5,100.0,"(Uuringu, 0, 2, None, None)",Uuringu,None
1434,348,Dev/ilu_orlau_ud211.conllu,Otse sealsamas üle oja .,15,22,üle oja,free,Otse sealsamas .,100.0,100.0,100.0,"(oja, 3, 2, None, None)",oja,None
1437,355,Train/tea_dr8047.conllu,"Merevetikate mõju munarebu ω-3-rasvhapete sisaldusele , mg/g rebus ( Herber , van Elswyk , 1996 )",61,97,"rebus ( Herber , van Elswyk , 1996 )",free,"Merevetikate mõju munarebu ω-3-rasvhapete sisaldusele , mg/g",85.7,85.7,85.7,"(rebus, 7, 7, None, None)",rebus,None
1512,507,Train/aja_horisont2000_osa_6_ud211.conllu,-. Uhke kaelkirjakupull Nairobi rahvuspargist Keenias .,46,53,Keenias,free,-. Uhke kaelkirjakupull Nairobi rahvuspargist .,0.0,0.0,33.3,"(Keenias, 5, 5, None, None)",Keenias,None


In [76]:
#data[data["verb"]=='ei']

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la,sentence_analysis_data,peasona,verb
193,409,Train/aja_ee199920_osa_1_ud211.conllu,""" Cannes'i kinnisvaramess ei ole tõsiselt võetav , see on mõeldud peamiselt riigiametnikele ilusa puhkuse veetmise kohaks , kuid mitte arvestatavate kontaktide loomiseks , "" lisab Slade .",92,169,"ilusa puhkuse veetmise kohaks , kuid mitte arvestatavate kontaktide loomiseks",free,""" Cannes'i kinnisvaramess ei ole tõsiselt võetav , see on mõeldud peamiselt riigiametnikele , "" lisab Slade .",100.0,100.0,100.0,"(,, 23, 7, ei, 4)",",",ei
398,898,Train/aja_ee199920_osa_2_ud211.conllu,Aga see informatsioon ei ole Eestis avalik .,29,35,Eestis,free,Aga see informatsioon ei ole avalik .,100.0,100.0,100.0,"(Eestis, 5, 7, ei, 4)",Eestis,ei
997,2294,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,Pikaajaline kortikosteroidravi ei ole oma rohkete kõrvaltoimete tõttu soovitatav .,38,69,oma rohkete kõrvaltoimete tõttu,free,Pikaajaline kortikosteroidravi ei ole soovitatav .,100.0,100.0,100.0,"(oma, 4, 7, ei, 3)",oma,ei
1046,2414,Train/aja_ee199920_osa_2_ud211.conllu,"Konkurentidest edukam ei oldud aga mitte šokolaadiga , vaid teerajajatena nendele tarbijatele orienteerudes , kes industrialiseerumisega oma toitumisharjumusi muutsid .",0,14,Konkurentidest,free,"edukam ei oldud aga mitte šokolaadiga , vaid teerajajatena nendele tarbijatele orienteerudes , kes industrialiseerumisega oma toitumisharjumusi muutsid .",57.9,57.9,78.9,"(Konkurentidest, 0, 2, ei, 3)",Konkurentidest,ei
1137,2607,Train/aja_pm20001004_osa_2_ud211.conllu,Avaliku arvamuse uuringute kvaliteet ja nende alusel esitatavad numbrid ei ole veel sotsioloogia .,40,52,nende alusel,free,Avaliku arvamuse uuringute kvaliteet ja esitatavad numbrid ei ole veel sotsioloogia .,100.0,100.0,100.0,"(nende, 5, 7, ei, 10)",nende,ei
